In [1]:
class EightPuzz:
    def __init__(self, initial_state, goal_state):
        self.initial_state = initial_state
        self.goal_state = goal_state
        self.closed_list = []

    def get_moves(self, state):
        possible_moves = []
        pos_x, pos_y = self.find_blank_coords(state)

        if pos_x > 0:
            possible_moves.append("up")
        if pos_x < 2:
            possible_moves.append("down")
        if pos_y > 0:
            possible_moves.append("left")
        if pos_y < 2:
            possible_moves.append("right")
        return possible_moves

    def execute_move(self, state, move):
        pos_x, pos_y = self.find_blank_coords(state)
        new_state = self.copy(state)

        if move == "up":
            new_state[pos_x][pos_y] = new_state[pos_x - 1][pos_y]
            new_state[pos_x - 1][pos_y] = state[pos_x][pos_y]
        elif move == "down":
            new_state[pos_x][pos_y] = new_state[pos_x + 1][pos_y]
            new_state[pos_x + 1][pos_y] = state[pos_x][pos_y]
        elif move == "left":
            new_state[pos_x][pos_y] = new_state[pos_x][pos_y - 1]
            new_state[pos_x][pos_y - 1] = state[pos_x][pos_y]
        elif move == "right":
            new_state[pos_x][pos_y] = new_state[pos_x][pos_y + 1]
            new_state[pos_x][pos_y + 1] = state[pos_x][pos_y]

        return new_state

    def successor_state_gen(self, state):
        successors = []
        possible_moves = self.get_moves(state)
        for moves in possible_moves:
            successors.append(self.execute_move(state, moves))
        print(successors)

    def find_blank_coords(self, state):
        for i in range(3):
            for j in range(3):
                if state[i][j] == 'B':
                    return i, j

    def copy(self, state):
        return [row[:] for row in state]

    def generate_path(self, closed_list):
        path = []
        path.append(closed_list[-1][1])
        current_state = closed_list[-1][1]
        for i in range(len(closed_list) - 1, -1, -1):
            if current_state == closed_list[i][1]:
                continue
            else:
                if current_state in self.successor_state_gen(closed_list[i][1]):
                    print(current_state)
                    path.append(closed_list[i][1])
                    current_state = closed_list[i][1]

In [2]:
class Heuristics:

    @staticmethod
    def hamming_distance(puzzle, successor):

        distance = 0
        for i in range(3):
            for j in range(3):
                if successor[i][j] != puzzle.goal_state[i][j]:
                    distance += 1
        return tuple([distance, successor])

    @staticmethod
    def manhattan_distance(puzzle, successor):
        distance = 0
        for i in range(3):
            for j in range(3):
                if successor[i][j] != 'B':
                    goal_row = (successor[i][j] - 1) // 3
                    goal_col = (successor[i][j] - 1) % 3
                else:
                    goal_row = 2
                    goal_col = 2
                distance += (abs(i - goal_row) + abs(j - goal_col))
        return tuple([distance, successor])

    @staticmethod
    def permutation_distance(puzzle, successor):

        distance = 0
        flattened = [element for sublist in successor for element in sublist]
        for i in range(9):
            for j in range(i + 1, 9):
                if flattened[i] == 'B':
                    distance += 1
                elif flattened[j] != 'B' and flattened[i] > flattened[j]:
                    distance += 1
        return tuple([distance, successor])
    @staticmethod
    def getInvCount(current_state):
        inversions = 0
        flattened = [element for sublist in current_state for element in sublist]
        for i in range(0, 9):
            for j in range(i + 1, 9):
                if flattened[i] != 'B' and flattened[j] != 'B' and flattened[i] > flattened[j]:
                    inversions += 1
        return inversions

    @staticmethod
    def inversion_manhattan(puzzle, successor):
        distance = 0
        for i in range(3):
            for j in range(3):
                if successor[i][j] != 'B':
                    goal_row = (successor[i][j] - 1) // 3
                    goal_col = (successor[i][j] - 1) % 3
                else:
                    goal_row = 2
                    goal_col = 2
                distance += (abs(i - goal_row) + abs(j - goal_col))
            distance += Heuristics.getInvCount(successor)
        return tuple([distance, successor])




In [3]:
def depth_first(puzzle):
    i = 0
    open_list = []
    closed_list = []
    start_state = puzzle.initial_state

    open_list = [start_state]

    while open_list:
        #         Replicates stack by popping last element put into list
        current_state = open_list.pop()

        if current_state == puzzle.goal_state:
            return i, current_state, "closed_list " + str(len(closed_list))

        else:
            for moves in puzzle.get_moves(current_state):
                successor_state = puzzle.execute_move(current_state, moves)
                closed_list.append(current_state)

                if successor_state not in closed_list and successor_state not in open_list:
                    open_list.append(successor_state)
        i = i + 1
    return None


def breadth_first(puzzle):
    i = 0
    open_list = []
    closed_list = []
    start_state = puzzle.initial_state

    open_list = [start_state]

    while open_list:
        #         Takes first element in the list.
        current_state = open_list.pop(0)

        if current_state == puzzle.goal_state:
            return i, current_state, "closed_list" + str(len(closed_list))

        else:
            for moves in puzzle.get_moves(current_state):
                successor_state = puzzle.execute_move(current_state, moves)
                closed_list.append(current_state)

                if successor_state not in closed_list and successor_state not in open_list:
                    open_list.append(successor_state)
        i = i + 1
    return None


# Uses heuristics so i will default it to hamming distance
def best_first(puzzle, heuristics):
    i = 0
    open_list = []
    closed_list = []
    start_state = puzzle.initial_state
    open_list = [tuple([1, start_state])]

    while open_list:
        open_list = sorted(open_list, key=lambda tup: tup[0])
        current_state_t = open_list.pop(0)

        current_state = current_state_t[1]

        if current_state == puzzle.goal_state:
            puzzle.closed_list = closed_list
            return current_state_t,i
        else:

            for moves in puzzle.get_moves(current_state):
                successor = puzzle.execute_move(current_state, moves)

                match heuristics:
                    case "hamming_distance":
                        successor_states = Heuristics.hamming_distance(puzzle, successor)
                    case "manhattan_distance":
                        successor_states = Heuristics.manhattan_distance(puzzle, successor)
                    case "permutation_distance":
                        successor_states = Heuristics.permutation_distance(puzzle, successor)
                    case "inversion_manhattan":
                        successor_states = Heuristics.inversion_manhattan(puzzle, successor)
                    case _:
                        successor_states = Heuristics.hamming_distance(puzzle, successor)

                if successor_states not in closed_list and successor_states not in open_list:
                    open_list.append(successor_states)
            closed_list.append(current_state_t)
            i=i+1
    return None


def a_star(puzzle, heuristics):
    closed_list = []
    start_state = puzzle.initial_state
    open_list = [tuple([1, start_state, 0])]
    i=0
    while open_list:
        open_list = sorted(open_list, key=lambda tup: tup[0])
        current_state_t = open_list.pop(0)

        current_state = current_state_t[1]
        depth = current_state_t[2]

        if current_state == puzzle.goal_state:
            puzzle.closed_list = closed_list
            return current_state_t,i


        else:
            execute = True
            for moves in puzzle.get_moves(current_state):
                state = puzzle.execute_move(current_state, moves)
                match heuristics:
                    case "hamming_distance":
                        successor_state = Heuristics.hamming_distance(puzzle, state)
                    case "manhattan_distance":
                        successor_state = Heuristics.manhattan_distance(puzzle, state)
                    case "permutation_distance":
                        successor_state = Heuristics.permutation_distance(puzzle, state)
                    case "inversion_manhattan":
                        successor_state = Heuristics.inversion_manhattan(puzzle, state)
                    case _:
                        successor_state = Heuristics.hamming_distance(puzzle, current_state)

                successor_state = tuple([(successor_state[0] + depth+ 1), successor_state[1], depth + 1])
#                 if state not in closed_list and state not in open_list:
#                     open_list.append(state)
#             closed_list.append(current_state_t)

                if not closed_list:
                    open_list.append(successor_state)

                #     remove closed states that have a higher cost
                to_remove = puzzle.copy(closed_list)
                for closed in to_remove:
                    if closed[1] == successor_state[1] and closed[0] > successor_state[0]:
                        open_list.append(successor_state)
                        closed_list.remove(closed)

                temp_open_list = puzzle.copy(open_list)
                for expanded in temp_open_list:
                    if expanded[1] == successor_state[1] and expanded[0] > successor_state[0]:
                        open_list.remove(expanded)
                        open_list.append(successor_state)
                        execute = False
                        break
                if execute:
                    open_list.append(successor_state)
        closed_list.append(current_state_t)
        i=i+1
    return None

    

In [15]:
initial_state = [[1,2,3],['B',4,6],[7,5,8]]
# initial_state = [[1,2,3],[4,5,6],[7,8,'B']]
goal_state = [[1,2,3],[4,5,6],[7,8,'B']]

puzzle = EightPuzz(initial_state,goal_state)

In [5]:
depth_first(puzzle)

(3527, [[1, 2, 3], [8, 'B', 4], [7, 6, 5]], 'closed_list 9966')

In [6]:
breadth_first(puzzle)

(34, [[1, 2, 3], [8, 'B', 4], [7, 6, 5]], 'closed_list94')

In [7]:
best_first(puzzle,"hamming_distance")

((0, [[1, 2, 3], [8, 'B', 4], [7, 6, 5]]), 5)

In [8]:
best_first(puzzle,"manhattan_distance")


((10, [[1, 2, 3], [8, 'B', 4], [7, 6, 5]]), 629)

In [9]:
best_first(puzzle,"permutation_distance")

((11, [[1, 2, 3], [8, 'B', 4], [7, 6, 5]]), 2976)

In [10]:
best_first(puzzle,"inversion_manhattan")

((31, [[1, 2, 3], [8, 'B', 4], [7, 6, 5]]), 8)

In [16]:
print(a_star(puzzle,"hamming_distance"))

((3, [[1, 2, 3], [4, 5, 6], [7, 8, 'B']], 3), 4)


In [17]:
print(a_star(puzzle,"manhattan_distance"))

((3, [[1, 2, 3], [4, 5, 6], [7, 8, 'B']], 3), 3)


In [18]:
print(a_star(puzzle,"permutation_distance"))

((3, [[1, 2, 3], [4, 5, 6], [7, 8, 'B']], 3), 5)


In [19]:
print(a_star(puzzle,"inversion_manhattan"))

((3, [[1, 2, 3], [4, 5, 6], [7, 8, 'B']], 3), 3)
